In [ ]:
!nvidia-smi

Mon Aug 14 04:12:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from io import BytesIO
from torchvision.transforms.functional import to_tensor, to_pil_image
import torch
from PIL import Image
import torch, gc
import glob
import os
gc.collect()
torch.cuda.empty_cache()

import ipywidgets as widgets
import IPython.display as display

def load_image(image_path, x32=False):
    img = Image.open(image_path).convert("RGB")

    if x32:
        def to_32s(x):
            return 256 if x < 256 else x - x % 32
        w, h = img.size
        img = img.resize((to_32s(w), to_32s(h)))

    return img
# input_path = "/content/input2/*"       create a new folder for input image and another for output
input_path = "/content/input/*"
o_path = "/content/output"
device = "cuda" if torch.cuda.is_available() else "cpu"
# model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
# face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
# model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", pretrained="paprika", device=device)
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", pretrained="face_paint_512_v2", device=device)



file_list = glob.glob(input_path)
print(file_list)
for image in file_list:
  img_name = os.path.basename(image)
  print(img_name)
  image = load_image(image)
  with torch.no_grad():
    image = to_tensor(image).unsqueeze(0) * 2 - 1
    out = model(image.to(device), False).cpu()
    out = out.squeeze(0).clip(-1, 1) * 0.5 + 0.5
    out = to_pil_image(out)
    output_path = os.path.join(o_path,img_name)
    out.save(output_path)
    print(f"image saved: {output_path}")

Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Downloading: "https://github.com/bryandlee/animegan2-pytorch/raw/main/weights/face_paint_512_v2.pt" to /root/.cache/torch/hub/checkpoints/face_paint_512_v2.pt
100%|██████████| 8.20M/8.20M [00:00<00:00, 176MB/s]


['/content/input/b2.png']
b2.png
image saved: /content/output/b2.png


In [ ]:

!pip install torch torchvision numpy
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5i8bhahb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-5i8bhahb
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch import nn
import numpy as np
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image
import clip


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 47.3MiB/s]


In [ ]:
input_image = Image.open("/content/pexels-анна-галашева-5742750.jpg")  # Provide the path to your image


In [ ]:
import torch
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image

# Define your stable diffusion model here
def your_diffusion_step(input_frame):
    # Replace this with your actual diffusion process
    # For this example, let's add Gaussian noise to the input frame
    noise = torch.randn_like(input_frame) * 0.1  # Adjust noise level as needed
    new_frame = input_frame + noise
    return new_frame

# Load the input image
input_image = Image.open("/content/pexels-анна-галашева-5742750.jpg")  # Provide the path to your image

# Preprocess the input image
preprocess = Compose([Resize((256, 256)), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
input_image = preprocess(input_image).unsqueeze(0)

# Generate animation frames using your stable diffusion model
num_frames = 20  # Adjust this based on your needs
animation_frames = []

current_frame = input_image
for _ in range(num_frames):
    new_frame = your_diffusion_step(current_frame)  # Replace with actual diffusion step

    animation_frames.append(to_pil_image(new_frame[0]))
    current_frame = new_frame

# Save the animation frames
for i, frame in enumerate(animation_frames):
    frame.save(f"animation_frame_{i}.png")



In [ ]:
def generate_animation_frames(input_image):
    # Define your stable diffusion model here
    # This function should generate a sequence of animation frames given an input image

    # For demonstration purposes, let's assume you have a loop that generates frames
    num_frames = 20  # Adjust this based on your needs
    animation_frames = []

    with torch.no_grad():
        current_frame = input_image
        for _ in range(num_frames):
            # Perform diffusion step to generate a new frame
            new_frame = your_diffusion_step(current_frame)  # Replace with actual diffusion step

            # Append the new frame to the list of animation frames
            animation_frames.append(new_frame)

            # Update the current frame for the next iteration
            current_frame = new_frame

    return animation_frames

# Load the input image
input_image = Image.open("/content/pexels-furknsaglam-4636160.jpg")  # Provide the path to your image

# Preprocess the input image
input_image = preprocess(input_image).unsqueeze(0).to(device)

# Generate animation frames using your stable diffusion model
animation_frames = generate_animation_frames(input_image)

# Save the animation frames
for i, frame in enumerate(animation_frames):
    frame_image = frame
    frame_image.save(f"animation_frame_{i}.png")




AttributeError: ignored

In [ ]:
import imageio
num_frames = 20

frames = []
for i in range(num_frames):  # Adjust 'num_frames' as needed
    frame = Image.open(f"animation_frame_{i}.png")
    frames.append(frame)

imageio.mimsave("animation3.gif", frames, duration=0.15)  # Adjust duration as needed


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image

# Load content and style images
content_image = Image.open("/content/pexels-furknsaglam-4636160.jpg")
style_image = Image.open("style.jpg")

# Preprocess images for VGG19 model
preprocess = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],      std=[0.229, 0.224, 0.225]),
])

content_tensor = preprocess(content_image).unsqueeze(0)
style_tensor = preprocess(style_image).unsqueeze(0)

# Load a pre-trained VGG19 model
vgg = models.vgg19(pretrained=True).features
vgg.to("cuda").eval()

# Define layers for style and content extraction
content_layers = ["conv_4"]
style_layers = ["conv_1", "conv_2", "conv_3", "conv_4", "conv_5"]

# Initialize the generated image as a copy of content image
generated_image = content_tensor.clone().requires_grad_(True).to("cuda")

# Hyperparameters
style_weight = 1000000
content_weight = 1
lr = 0.01
steps = 2000

optimizer = optim.Adam([generated_image], lr=lr)

# Style transfer process
for step in range(steps):
    optimizer.zero_grad()

    # Pass the generated image through the VGG19 model
    model_output = vgg(generated_image)

    content_loss = 0
    style_loss = 0

    for layer_name, target_content, target_style in zip(content_layers, content_tensor, style_tensor):
        layer = model_output[layer_name]

        content_loss += torch.mean((layer - target_content.to("cuda")) ** 2)

        _, c, h, w = layer.shape
        layer = layer.view(c, h * w)
        target_style = target_style.view(c, h * w)

        gram_matrix = torch.mm(layer, layer.t())
        target_gram_matrix = torch.mm(target_style, target_style.t())

        style_loss += torch.mean((gram_matrix - target_gram_matrix.to("cuda")) ** 2)

    total_loss = content_weight * content_loss + style_weight * style_loss
    total_loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Step [{step}/{steps}], Loss: {total_loss.item()}")

# Convert the generated image tensor back to a PIL image
generated_image = generated_image.cpu().detach().squeeze(0)
generated_image = transforms.ToPILImage()(generated_image)

# Save the final generated image
generated_image.save("generated_image.jpg")
